### Load Data

In [192]:
import pandas as pd

In [193]:
links = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/links.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [194]:
movies = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/movies.csv')

In [195]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [196]:
ratings = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/ratings.csv')


In [197]:
tags = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/tags.csv')

In [198]:
ratings = ratings.drop('timestamp', axis=1)

In [199]:
ratings.shape

(100836, 3)

In [200]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [201]:
new = pd.merge(links, movies, on='movieId')

In [202]:
new_2 = pd.merge(new, ratings, on='movieId')

In [203]:
new_3 = pd.merge(new_2, tags, on='movieId')

### Merge Data Frames

In [204]:
# merge links and movie
#frames_1 = [links, movies]
#new = pd.concat(frames_1)


In [205]:
# merge new and ratings
#frames_2 = [new, ratings]
#new_2 = pd.concat(frames_2)

In [206]:
# Drop movieId and timestamp
#tags_2 = tags.drop(columns=['movieId', 'timestamp'])

In [207]:
# concat new_2 and tags
#frames = [new_2, tags_2]
#new_3 = pd.concat(frames)
#new_3 = pd.merge(new_2, tags)


In [208]:
new_3.shape

(233213, 10)

### Preprocessing

In [209]:
# Check for null values
new_3.isnull().sum()

movieId      0
imdbId       0
tmdbId       0
title        0
genres       0
userId_x     0
rating       0
userId_y     0
tag          0
timestamp    0
dtype: int64

In [210]:
new_3['movieId'].nunique()

1554

In [211]:
# replace | with , in the genres column
new_3['genres'] = new_3['genres'].str.replace('|',' , ')

In [212]:
#df = new_3.drop_duplicates(subset=['movieId'])

In [213]:
# convert df to a csv file
new_3.to_csv('/Users/gabrielwarner/Data-Science/Projects/phase_4/movies_2.csv')

### Explode genres column

In [214]:
# split genre column based on ,
new_3['new_genres'] = new_3['genres'].str.split(',', 10)

In [215]:
# explode new_genres
df = new_3.explode('new_genres')

In [216]:
# drop genres from df
df_2 = df.drop(columns=['genres'])

In [217]:
df_2.head()

,movieId,imdbId,tmdbId,title,userId_x,rating,userId_y,tag,timestamp,new_genres
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Adventure
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Animation
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Children
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Comedy
0,1,114709,862.0,Toy Story (1995),1,4.0,336,pixar,1139045764,Fantasy


### OHE title

In [218]:
one_hot = pd.get_dummies(df_2['title'])

In [219]:
df = df_2.drop('title',axis = 1)

In [220]:
df = pd.concat([df, one_hot], axis=1)

In [221]:
df.shape

(765271, 1563)

In [222]:
one_hot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 1554 entries, (500) Days of Summer (2009) to eXistenZ (1999)
dtypes: uint8(1554)
memory usage: 1.1 GB


### OHE tag

In [223]:
one_hot_tag = pd.get_dummies(df['tag'])

In [224]:
df = df.drop('tag',axis = 1)

In [225]:
df = pd.concat([df, one_hot_tag], axis=1)

In [226]:
one_hot_tag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 1584 entries, "artsy" to zombies
dtypes: uint8(1584)
memory usage: 1.1 GB


### OHE new_genres

In [227]:
one_hot_genre = pd.get_dummies(df['new_genres'])

In [228]:
df =df.drop('new_genres',axis = 1)

In [229]:
df = pd.concat([df, one_hot_genre], axis=1)

### Base Line Model

In [39]:
from sklearn.neighbors import NearestNeighbors

In [40]:
#knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
#knn.fit(df)

### Suprise

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765271 entries, 0 to 233212
Columns: 3209 entries, movieId to Western
dtypes: float64(2), int64(5), uint8(3202)
memory usage: 2.3 GB


### Change Data Frames to suprise DataFrame

In [230]:
import surprise

In [231]:
from surprise.model_selection import train_test_split
# from sklearn.model_selection import train_test_split

In [232]:
ratings.rating.describe()

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [233]:
ratings.shape

(100836, 3)

In [234]:
reader = surprise.Reader(rating_scale = (4., 5.))

In [235]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [236]:
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [237]:
data = surprise.Dataset.load_from_df(ratings, reader)

In [238]:
df.head()

,movieId,imdbId,tmdbId,userId_x,rating,userId_y,timestamp,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),...,Musical,Musical,Mystery,Mystery,Romance,Romance,Sci-Fi,Sci-Fi,Thriller,Western
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,114709,862.0,1,4.0,336,1139045764,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [239]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [240]:
# from sklearn.model_selection import train_test_split

In [241]:
trainset, testset = train_test_split(data, test_size=0.2)

In [242]:
#trainset.head()

### KNNS Basic (Best RMSE=.5)

In [55]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

In [56]:
basic = knns.KNNBasic()
basic.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [57]:
predictions = basic.test(testset)

In [58]:
print(accuracy.rmse(predictions))

RMSE: 1.1490
1.1490105459461224


In [59]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1496
1.1495873434523884


### Matrix Factorization

In [60]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

### Suprise with grid search

In [61]:
from sklearn.neighbors import KNeighborsClassifier

In [62]:
param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
               'reg_all': [0.4, 0.6]}

In [63]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors':[20, 50, 75, 100],'n_epochs': [5, 6, 7, 8, 9, 10, 11], 'lr_all': [0.002, .003, .004, 0.005],
               'reg_all': [.02, 0.4, .5, 0.6]}
gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
gs_model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   51.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 15.0min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 2240 out of 2240 | elapsed: 19.4min finished


In [64]:
svd = SVD(n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.1449
1.1449304947653431


### Predictions SVD

In [65]:
user_34_prediction = svd.predict('12', '85')
user_34_prediction

Prediction(uid='12', iid='85', r_ui=None, est=4.0, details={'was_impossible': False})

In [66]:
user_34_prediction[3]

4.0

In [67]:
def get_Iu(uid):
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
def get_Ui(iid):
    """
args:
iid: the raw id of the item
returns:
the number of users that have rated the item.
"""
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
df_predictions = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_predictions['Iu'] = df_predictions.uid.apply(get_Iu)
df_predictions['Ui'] = df_predictions.iid.apply(get_Ui)
df_predictions['err'] = abs(df_predictions.est - df_predictions.rui)

In [68]:
best_predictions = df_predictions.sort_values(by='err')[:10]
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
0,391,1094,4.0,4.0,{'was_impossible': False},298,31,0.0
6571,566,21,4.0,4.0,{'was_impossible': False},57,68,0.0
6568,64,596,4.0,4.0,{'was_impossible': False},394,47,0.0
16312,64,4027,4.0,4.0,{'was_impossible': False},394,73,0.0
6558,551,68954,4.0,4.0,{'was_impossible': False},97,85,0.0
16315,514,3552,4.0,4.0,{'was_impossible': False},322,39,0.0
16316,68,44665,4.0,4.0,{'was_impossible': False},997,28,0.0
16317,96,3176,4.0,4.0,{'was_impossible': False},69,39,0.0
6549,414,1092,4.0,4.0,{'was_impossible': False},2182,36,0.0
16320,356,8784,4.0,4.0,{'was_impossible': False},233,41,0.0


### Graph Lab Model

In [69]:
# Make a train-test split
#train_data, validate_data = df.random_split(0.8)

### Cosine Similarity

In [70]:
from surprise import KNNBasic
from surprise.model_selection import cross_validate

In [71]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

In [72]:
#knn.train(trainset)

### Surprise Baseline Only

In [73]:
from surprise import BaselineOnly

In [74]:
# showcasing the cross_validate function as well 
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1622  1.1463  1.1452  1.1435  1.1482  1.1491  0.0067  
MAE (testset)     0.8495  0.8386  0.8371  0.8289  0.8357  0.8380  0.0067  
Fit time          0.21    0.23    0.23    0.23    0.23    0.23    0.01    
Test time         0.08    0.37    0.08    0.08    0.08    0.13    0.12    


{'test_rmse': array([1.16223423, 1.14630469, 1.14523288, 1.14348123, 1.14818636]),
 'test_mae': array([0.84954504, 0.83862824, 0.83712275, 0.82886079, 0.83574079]),
 'fit_time': (0.21460485458374023,
  0.23051095008850098,
  0.23468971252441406,
  0.22982406616210938,
  0.23163318634033203),
 'test_time': (0.0761561393737793,
  0.36571598052978516,
  0.0769493579864502,
  0.0761420726776123,
  0.07587790489196777)}

### KNN with GridSearch

In [75]:
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV

In [ ]:

sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [2, 3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs_1 = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs_1.fit(data)

print(gs_1.best_score["rmse"])
print(gs_1.best_params["rmse"])

### Grid Search

In [77]:
# param_grid = {
        #"n_epochs": [5, 10, 15, 20, 30, 40, 50, 100],
        # "lr_all": [0.001, 0.002, 0.005],
       # "reg_all": [0.02, 0.08, 0.4, 0.6]
#}

# smaller grid for testing
param_grid = {
    "n_epochs": [15, 20, 25],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.02]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=5)

gs.fit(data)

training_parameters = gs.best_params["rmse"]

print("BEST RMSE: \t", gs.best_score["rmse"])
print("BEST MAE: \t", gs.best_score["mae"])
print("BEST params: \t", gs.best_params["rmse"])

BEST RMSE: 	 1.1493286306633104
BEST MAE: 	 0.8373425298930179
BEST params: 	 {'n_epochs': 15, 'lr_all': 0.005, 'reg_all': 0.02}


In [78]:
def get_movie_id(movie_title):
    
    # Filter Movies dataframe for movie_title
    movie_title_df = movies[['title', 'movieId']]
    # Get corresponding id from row
    #movie_id = movie_title_df.query('title==movie_title')['movieId']
    movie_id = movie_title_df.loc['movieId']
    # Return movie_id of movie_title
    return movie_id

8983

In [179]:
#movies = movies.set_index('title')

In [252]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [260]:
def get_movie_id(movie_title):
    
    # Filter Movies dataframe for movie_title
    movie_title_df = movies[['title', 'movieId']]
    # Get corresponding id from row
    #movie_id = movie_title_df.query('title==movie_title')['movieId']
  #  movie_id = movie_title_df['movieId']
    movie_id = movies[movies["title"] == movie_title]['movieId'].values[0]

    # Return movie_id of movie_title
    return movie_id


In [272]:
t = pd.DataFrame([[1, 5, 4]], columns=['u', 'i', 'rating'])
t

,u,i,rating
0,1,5,4


In [269]:
movies_id = get_movie_id('Father of the Bride Part II (1995)')
# pass movie_id and a user_id to model to make recs
gs.test(testset)
gs.predict()

[Prediction(uid=380, iid=172887, r_ui=3.0, est=4.0, details={'was_impossible': False}),
 Prediction(uid=563, iid=7444, r_ui=3.5, est=4.0, details={'was_impossible': False}),
 Prediction(uid=599, iid=1261, r_ui=3.0, est=4.0, details={'was_impossible': False}),
 Prediction(uid=141, iid=6863, r_ui=3.0, est=4.0, details={'was_impossible': False}),
 Prediction(uid=64, iid=3969, r_ui=0.5, est=4.0, details={'was_impossible': False}),
 Prediction(uid=298, iid=2502, r_ui=3.0, est=4.0, details={'was_impossible': False}),
 Prediction(uid=387, iid=7450, r_ui=3.0, est=4.0, details={'was_impossible': False}),
 Prediction(uid=122, iid=480, r_ui=4.0, est=4.536912604095614, details={'was_impossible': False}),
 Prediction(uid=3, iid=5764, r_ui=4.5, est=4.0, details={'was_impossible': False}),
 Prediction(uid=448, iid=2111, r_ui=4.0, est=4.0, details={'was_impossible': False}),
 Prediction(uid=601, iid=2355, r_ui=4.5, est=4.1943974634469, details={'was_impossible': False}),
 Prediction(uid=450, iid=1580,

In [265]:
get_movie_id('Father of the Bride Part II (1995)')

5

In [83]:
#top_n('Toy Story 1995')

In [108]:
#get_top_n('Toy Story (1995)')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Movie Reccomendation Model

In [ ]:
df_2

In [ ]:
df_2.head()

In [ ]:
import RAKE
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df_3 = df_2[['title','new_genres','tag','rating']]
df_3.head()

In [ ]:
# initializing the new column
df_3['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['tag']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df_3.drop(columns = ['tag'], inplace = True)

In [ ]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['tag'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [ ]:
recommendations('Toy Story (1995)')

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

### My Own Algorythem

In [ ]:
ratings.head(1)

In [ ]:
from surprise import AlgoBase
from surprise.model_selection import cross_validate


class MyOwnAlgorithm(AlgoBase):

    def __init__(self):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)

    def estimate(self, u, i):

        return 3


#data = Dataset.load_builtin('ml-100k')
algo = MyOwnAlgorithm()

cross_validate(algo, data, verbose=True)

In [ ]:
class MyOwnAlgorithm(AlgoBase):

    def __init__(self):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)

    def fit(self, trainset):

        # Here again: call base method before doing anything.
        AlgoBase.fit(self, trainset)

        # Compute the average rating. We might as well use the
        # trainset.global_mean attribute ;)
        self.the_mean = np.mean([r for (_, _, r) in
                                 self.trainset.all_ratings()])

        return self

    def estimate(self, u, i):

        return self.the_mean

In [ ]:
    def estimate(self, u, i):

        sum_means = self.trainset.global_mean
        div = 1

        if self.trainset.knows_user(u):
            sum_means += np.mean([r for (_, r) in self.trainset.ur[u]])
            div += 1
        if self.trainset.knows_item(i):
            sum_means += np.mean([r for (_, r) in self.trainset.ir[i]])
            div += 1

        return sum_means / div

In [ ]:
from surprise import PredictionImpossible

In [ ]:
class MyOwnAlgorithm(AlgoBase):

    def __init__(self, sim_options={}, bsl_options={}):

        AlgoBase.__init__(self, sim_options=sim_options,
                          bsl_options=bsl_options)

    def fit(self, trainset):

        AlgoBase.fit(self, trainset)

        # Compute baselines and similarities
        self.bu, self.bi = self.compute_baselines()
        self.sim = self.compute_similarities()

        return self

    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unknown.')

        # Compute similarities between u and v, where v describes all other
        # users that have also rated item i.
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print('The 3 nearest neighbors of user', str(u), 'are:')
        for v, sim_uv in neighbors[:3]:
            print('user {0:} with sim {1:1.2f}'.format(v, sim_uv))

        # ... Aaaaand return the baseline estimate anyway ;)

In [ ]:
import inspect

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for userId, movieId, rating in predictions.items():
        top_n[userId].append((movieId, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for userId, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[userId] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=3)

# Print the recommended items for each user
#for uid, user_ratings in top_n.items():
   # print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
get_top_n('Toy Story (1995)')

In [ ]:
ratings

In [ ]:
movies

In [ ]:
#get_top_n('Toy Story (1995)')

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    #top_n = defaultdict(list)
    #for uid, iid, true_r, est, _ in predictions:
        #top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
#data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
#for uid, user_ratings in top_n.items():
   # print(uid, [iid for (iid, _) in user_ratings])